<a href="https://colab.research.google.com/github/amine406/SeoProject/blob/main/Crawling_asynchrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Initialiser les variables
visited_urls = set()  # Pour stocker les URLs visitées
url_info_list = []     # Stocker les infos SEO pour chaque URL

# Créer une session globale
session = requests.Session()

# Ajouter un User-Agent à la session
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
})

# Fonction pour vérifier si l'URL est valide et interne
def is_valid_url(url, base_url):
    parsed_url = urlparse(url)
    return bool(parsed_url.scheme) and bool(parsed_url.netloc) and base_url in url

# Fonction pour crawler et analyser une page
def crawl_page(url, base_url):
    try:
        response = session.get(url, timeout=10)  # Timeout après 10 secondes
        status_code = response.status_code
        print(f"Crawling {url} {status_code}")

        if status_code != 200:
            return []  # Ignorer si le statut n'est pas OK

        # Vérifier que la réponse est bien du HTML
        content_type = response.headers.get("Content-Type", "")
        if "text/html" not in content_type:
            return []  # Ignorer le contenu non-HTML

        # Utiliser un parser robuste
        soup = BeautifulSoup(response.content, "html.parser")

        # Récupérer les informations SEO
        title = soup.title.string.strip() if soup.title and soup.title.string else "No title"
        description_tag = soup.find("meta", attrs={"name": "description"})
        description = description_tag["content"].strip() if description_tag and description_tag.get("content") else "No description"
        h1_tag = soup.find("h1")
        h1_text = h1_tag.get_text(strip=True) if h1_tag else "No H1"

        # Compter le nombre de liens sur la page
        links = soup.find_all("a", href=True)
        num_links = len(links)

        # Trouver les nouveaux liens internes
        new_links = [
            urljoin(base_url, link.get('href'))
            for link in links
            if is_valid_url(urljoin(base_url, link.get('href')), base_url)
        ]

        # Message des nouveaux liens trouvés
        message_nouveaux_liens = f"Nouveaux liens trouvés :\n" + "\n".join(new_links)

        # Ajouter les informations SEO et les nouveaux liens au tableau
        url_info_list.append({
            "URL": url,
            "HTTP Status": status_code,
            "Title": title,
            "Meta Description": description,
            "H1": h1_text,
            "Number of Links": num_links,
            "New Links Found": "\n".join(new_links),  # Retour à la ligne pour chaque nouveau lien
            "Message Nouveaux Liens": message_nouveaux_liens
        })

        # Retourner les nouveaux liens pour le crawling
        return new_links

    except requests.RequestException as e:
        print(f"Failed to crawl {url}: {e}")
        return []  # Retourner une liste vide en cas d'erreur

# Fonction principale pour démarrer le crawling sans limitation de profondeur avec parallélisation
def crawl_website(start_url, max_workers=10):
    base_url = "{0.scheme}://{0.netloc}".format(urlparse(start_url))
    to_crawl = [start_url]  # Liste des URLs à crawler
    visited_urls.add(start_url)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Dictionnaire pour suivre les futures
        future_to_url = {executor.submit(crawl_page, url, base_url): url for url in to_crawl}

        while future_to_url:
            # Attendre que les futures soient terminées
            done, _ = as_completed(future_to_url), None
            for future in done:
                url = future_to_url[future]
                try:
                    new_links = future.result()
                    print(f"Nouveaux liens trouvés : {new_links}")

                    # Ajouter les nouveaux liens à la liste des URLs à crawler
                    for link in new_links:
                        if link not in visited_urls:
                            visited_urls.add(link)
                            future_to_url[executor.submit(crawl_page, link, base_url)] = link

                except Exception as e:
                    print(f"Error processing {url}: {e}")

                # Retirer le future terminé du dictionnaire
                del future_to_url[future]
                break  # Recommencer la boucle while

    # Convertir les données collectées en DataFrame
    df = pd.DataFrame(url_info_list)

    # Afficher le tableau
    print("\nTableau récapitulatif des pages crawlées:")
    print(df)

    # Exporter vers un fichier CSV si des données ont été collectées
    if not df.empty:
        df.to_csv("seo_crawl_results_file3.csv", index=False)
        print("Résultats exportés vers 'seo_crawl_results_file3.csv'.")
    else:
        print("Aucune donnée à exporter.")

    # Démarrer le crawler sur le site "https://aundetailpres.fr"
if __name__ == "__main__": # removed extra indent
    print("Démarrage du crawling sur https://aundetailpres.fr...")
    crawl_website("https://aundetailpres.fr")
    print("Crawling terminé.")